# Fase B: Construcción del Dataset a Nivel Producto
## Preparación de datos para clustering de productos

### Paso 1: Importación de librerías y carga de datos

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
import warnings
warnings.filterwarnings('ignore')

# Cargar datos
df = pd.read_csv('../Data/Raw/train.csv')

### Paso 2: Limpieza de Item_Fat_Content

In [2]:
# Estandarizar valores de Item_Fat_Content
fat_content_mapping = {
    'Low Fat': 'Low Fat',
    'low fat': 'Low Fat', 
    'LF': 'Low Fat',
    'Regular': 'Regular',
    'reg': 'Regular'
}

df['Item_Fat_Content'] = df['Item_Fat_Content'].map(fat_content_mapping)
print("Valores únicos en Item_Fat_Content después de limpieza:")
print(df['Item_Fat_Content'].value_counts())

Valores únicos en Item_Fat_Content después de limpieza:
Item_Fat_Content
Low Fat    5517
Regular    3006
Name: count, dtype: int64


### Paso 3: Tratamiento inteligente de valores nulos

In [3]:
# Para Item_Weight: Buscar el peso del mismo Item_Identifier en otras filas
def fill_item_weight(row):
    if pd.isna(row['Item_Weight']):
        same_item_weights = df[df['Item_Identifier'] == row['Item_Identifier']]['Item_Weight']
        same_item_weights = same_item_weights.dropna()
        if len(same_item_weights) > 0:
            return same_item_weights.iloc[0]
    return row['Item_Weight']

df['Item_Weight'] = df.apply(fill_item_weight, axis=1)

# Para visibilidad 0: tratarlos como nulos e imputar con promedio del producto
df['Item_Visibility'] = df['Item_Visibility'].replace(0, np.nan)

def fill_item_visibility(row):
    if pd.isna(row['Item_Visibility']):
        same_item_visibility = df[df['Item_Identifier'] == row['Item_Identifier']]['Item_Visibility']
        same_item_visibility = same_item_visibility.dropna()
        if len(same_item_visibility) > 0:
            return same_item_visibility.mean()
    return row['Item_Visibility']

df['Item_Visibility'] = df.apply(fill_item_visibility, axis=1)

# Para los nulos restantes, usar mediana por tipo de producto
weight_imputer = SimpleImputer(strategy='median')
df['Item_Weight'] = weight_imputer.fit_transform(df[['Item_Weight']])

visibility_imputer = SimpleImputer(strategy='median')
df['Item_Visibility'] = visibility_imputer.fit_transform(df[['Item_Visibility']])

print("Resumen de nulos después del tratamiento:")
print(df.isnull().sum())

Resumen de nulos después del tratamiento:
Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  2410
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales               0
dtype: int64


### Paso 4: Feature Engineering - Crear categorías amplias

In [4]:
# Crear categoría amplia basada en las primeras dos letras del ID
def get_broad_category(item_id):
    prefix = item_id[:2]
    if prefix == 'FD':
        return 'Food'
    elif prefix == 'DR':
        return 'Drink'
    elif prefix == 'NC':
        return 'Non-Consumable'
    else:
        return 'Other'

df['Item_Category_Broad'] = df['Item_Identifier'].apply(get_broad_category)

# Ajustar Fat_Content para productos no consumibles
df.loc[df['Item_Category_Broad'] == 'Non-Consumable', 'Item_Fat_Content'] = 'Not Applicable'

print("Distribución de categorías amplias:")
print(df['Item_Category_Broad'].value_counts())
print("\nDistribución de Fat_Content después del ajuste:")
print(df['Item_Fat_Content'].value_counts())

Distribución de categorías amplias:
Item_Category_Broad
Food              6125
Non-Consumable    1599
Drink              799
Name: count, dtype: int64

Distribución de Fat_Content después del ajuste:
Item_Fat_Content
Low Fat           3918
Regular           3006
Not Applicable    1599
Name: count, dtype: int64


### Paso 5: Construcción del dataset agregado a nivel producto

In [5]:
# Agrupar por producto y calcular métricas agregadas
product_aggregated = df.groupby('Item_Identifier').agg({
    'Item_Outlet_Sales': ['sum', 'mean'],
    'Outlet_Identifier': 'nunique',
    'Item_MRP': 'mean',
    'Item_Visibility': 'mean',
    'Item_Weight': 'mean',
    'Item_Fat_Content': 'first',
    'Item_Type': 'first',
    'Item_Category_Broad': 'first'
}).reset_index()

# Renombrar columnas para mayor claridad
product_aggregated.columns = [
    'Item_Identifier',
    'Total_Sales',
    'Avg_Sales',
    'Store_Count',
    'Avg_MRP',
    'Avg_Visibility',
    'Item_Weight',
    'Item_Fat_Content',
    'Item_Type',
    'Item_Category_Broad'
]

# Calcular métricas adicionales
product_aggregated['Sales_Per_Store'] = product_aggregated['Total_Sales'] / product_aggregated['Store_Count']
product_aggregated['Price_Per_Unit_Weight'] = product_aggregated['Avg_MRP'] / product_aggregated['Item_Weight']

print(f"Dataset agregado: {product_aggregated.shape}")
print("\nPrimeras filas del dataset agregado:")
product_aggregated.head()

Dataset agregado: (1559, 12)

Primeras filas del dataset agregado:


,Item_Identifier,Total_Sales,Avg_Sales,Store_Count,Avg_MRP,Avg_Visibility,Item_Weight,Item_Fat_Content,Item_Type,Item_Category_Broad,Sales_Per_Store,Price_Per_Unit_Weight
0,DRA12,11061.6012,1843.600200,6,141.865400,0.047934,11.600,Low Fat,Soft Drinks,Drink,1843.600200,12.229776
1,DRA24,15723.5328,2246.218971,7,164.086800,0.048062,19.350,Regular,Soft Drinks,Drink,2246.218971,8.479938
2,DRA59,20915.4412,2614.430150,8,185.179900,0.153963,8.270,Regular,Soft Drinks,Drink,2614.430150,22.391765
3,DRB01,4554.0720,1518.024000,3,189.586333,0.082126,7.390,Low Fat,Soft Drinks,Drink,1518.024000,25.654443
4,DRB13,12144.1920,2428.838400,5,189.693000,0.008002,6.115,Regular,Soft Drinks,Drink,2428.838400,31.020932


### Paso 6: Encoding y Scaling para clustering

In [6]:
# Crear copia para interpretación (sin escalar)
product_interpretation = product_aggregated.copy()

# Seleccionar variables para clustering
clustering_vars = [
    'Total_Sales', 'Avg_Sales', 'Store_Count', 'Avg_MRP', 
    'Avg_Visibility', 'Item_Weight', 'Sales_Per_Store', 'Price_Per_Unit_Weight'
]

# Aplicar log-transform a variables sesgadas
skewed_vars = ['Total_Sales', 'Avg_Sales', 'Avg_Visibility']
for var in skewed_vars:
    product_aggregated[f'log_{var}'] = np.log1p(product_aggregated[var])

# Actualizar variables de clustering
clustering_vars_transformed = [f'log_{var}' if var in skewed_vars else var for var in clustering_vars]

# Escalar variables
scaler = StandardScaler()
scaled_features = scaler.fit_transform(product_aggregated[clustering_vars_transformed])

# Crear dataset para modelado
product_modeling = pd.DataFrame(scaled_features, columns=clustering_vars_transformed)
product_modeling['Item_Identifier'] = product_aggregated['Item_Identifier']

# Encoding de variables categóricas para análisis
label_encoders = {}
categorical_vars = ['Item_Fat_Content', 'Item_Type', 'Item_Category_Broad']

for var in categorical_vars:
    le = LabelEncoder()
    product_interpretation[f'{var}_encoded'] = le.fit_transform(product_interpretation[var])
    label_encoders[var] = le

print("Dataset para modelado (primeras filas):")
print(product_modeling.head())
print(f"\nShape del dataset para modelado: {product_modeling.shape}")

Dataset para modelado (primeras filas):
   log_Total_Sales  log_Avg_Sales  Store_Count   Avg_MRP  log_Avg_Visibility  \
0         0.182047      -0.026613     0.348838  0.013769           -0.463337   
1         0.698677       0.302109     1.003278  0.371701           -0.460515   
2         1.117845       0.554748     1.657717  0.711459            1.763570   
3        -1.121573      -0.349946    -1.614480  0.782436            0.278502   
4         0.319212       0.432198    -0.305601  0.784154           -1.360979   

   Item_Weight  Sales_Per_Store  Price_Per_Unit_Weight Item_Identifier  
0    -0.260117        -0.308001              -0.068270           DRA12  
1     1.408477         0.047767              -0.534033           DRA24  
2    -0.977074         0.373131               1.193937           DRA59  
3    -1.166540        -0.595691               1.599190           DRB01  
4    -1.441051         0.209136               2.265755           DRB13  

Shape del dataset para modelado: (1559, 

### Paso 7: Guardar datasets procesados

In [7]:
# Guardar datasets
product_modeling.to_csv('../Data/Processed/product_modeling_dataset_deep.csv', index=False)
product_interpretation.to_csv('../Data/Processed/product_interpretation_dataset_deep.csv', index=False)

print("✓ Datasets guardados exitosamente:")
print("  - product_modeling_dataset_deep.csv (para clustering)")
print("  - product_interpretation_dataset_deep.csv (para análisis)")

✓ Datasets guardados exitosamente:
  - product_modeling_dataset_deep.csv (para clustering)
  - product_interpretation_dataset_deep.csv (para análisis)


### Paso 8: Resumen de transformaciones aplicadas

In [8]:
print("=== RESUMEN DE TRANSFORMACIONES APLICADAS ===\n")

transformations = [
    ("Limpieza de Item_Fat_Content", "Estandarización de valores inconsistentes (Low Fat, LF, low fat → Low Fat; Regular, reg → Regular)"),
    ("Tratamiento de Item_Weight nulos", "Imputación usando valores del mismo producto, luego mediana por tipo de producto"),
    ("Tratamiento de Item_Visibility cero", "Valores 0 tratados como nulos e imputados con promedio del mismo producto"),
 ("Feature Engineering", "Creación de Item_Category_Broad (Food/Drink/Non-Consumable) basado en prefijos del ID"),
    ("Ajuste lógico", "Fat_Content marcado como 'Not Applicable' para productos no consumibles"),
    ("Agregación a nivel producto", "Cálculo de 8 métricas clave por producto incluyendo ventas totales, promedio y penetración"),
    ("Transformación de variables", "Aplicación de log-transform a variables sesgadas (Total_Sales, Avg_Sales, Avg_Visibility)"),
    ("Escalado", "Normalización StandardScaler para todas las variables numéricas"),
    ("Encoding", "Label encoding para variables categóricas en dataset de interpretación"),
    ("Creación de datasets", "2 datasets: product_modeling_dataset (scaled) y product_interpretation_dataset (raw + encoded)")
]

for i, (step, description) in enumerate(transformations, 1):
    print(f"{i}. {step}: {description}")

print(f"\n=== RESULTADOS FINALES ===")
print(f"• Productos únicos procesados: {len(product_aggregated)}")
print(f"• Variables para clustering: {len(clustering_vars_transformed)}")
print(f"• Variables categóricas codificadas: {len(categorical_vars)}")
print(f"• Datasets generados: 2 (modelado + interpretación)")

=== RESUMEN DE TRANSFORMACIONES APLICADAS ===

1. Limpieza de Item_Fat_Content: Estandarización de valores inconsistentes (Low Fat, LF, low fat → Low Fat; Regular, reg → Regular)
2. Tratamiento de Item_Weight nulos: Imputación usando valores del mismo producto, luego mediana por tipo de producto
3. Tratamiento de Item_Visibility cero: Valores 0 tratados como nulos e imputados con promedio del mismo producto
4. Feature Engineering: Creación de Item_Category_Broad (Food/Drink/Non-Consumable) basado en prefijos del ID
5. Ajuste lógico: Fat_Content marcado como 'Not Applicable' para productos no consumibles
6. Agregación a nivel producto: Cálculo de 8 métricas clave por producto incluyendo ventas totales, promedio y penetración
7. Transformación de variables: Aplicación de log-transform a variables sesgadas (Total_Sales, Avg_Sales, Avg_Visibility)
8. Escalado: Normalización StandardScaler para todas las variables numéricas
9. Encoding: Label encoding para variables categóricas en dataset d